# code from scratch

# 1. NICE (Non-linear Independent Components Estimation)


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the coupling layer for NICE
class AdditiveCoupling(layers.Layer):
    def __init__(self):
        super(AdditiveCoupling, self).__init__()

    def call(self, x):
        x1, x2 = tf.split(x, num_or_size_splits=2, axis=-1)
        y1 = x1
        y2 = x2 + tf.nn.tanh(x1)  # Simple additive function
        return tf.concat([y1, y2], axis=-1)

    def inverse(self, y):
        y1, y2 = tf.split(y, num_or_size_splits=2, axis=-1)
        x1 = y1
        x2 = y2 - tf.nn.tanh(y1)
        return tf.concat([x1, x2], axis=-1)

# Define the NICE model with multiple coupling layers
class NICEModule(keras.Model):
    def __init__(self):
        super(NICEModule, self).__init__()
        self.layers_ = [AdditiveCoupling() for _ in range(4)]  # Stack 4 layers

    def call(self, x):
        for layer in self.layers_:
            x = layer(x)
        return x

    def inverse(self, y):
        for layer in reversed(self.layers_):
            y = layer.inverse(y)
        return y

# Create the NICE model
input_shape = (28, 28, 1)  # Example input shape (e.g., MNIST)
nice_model = NICEModule()

# Forward pass (data transformation)
x = tf.random.normal((32, 28, 28, 1))  # Batch of 32 samples
y = nice_model(x)

# Inverse pass (sampling)
x_reconstructed = nice_model.inverse(y)


InvalidArgumentError: Exception encountered when calling AdditiveCoupling.call().

[1m{{function_node __wrapped__Split_num_split_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Number of ways to split should evenly divide the split dimension, but got split_dim 3 (size = 1) and num_split 2 [Op:Split] name: split[0m

Arguments received by AdditiveCoupling.call():
  • x=tf.Tensor(shape=(32, 28, 28, 1), dtype=float32)

# 2. RealNVP (Real-valued Non-Volume Preserving)


In [ ]:
import tensorflow_probability as tfp
tfd = tfp.distributions
tfpl = tfp.layers

# Define the affine coupling layer for RealNVP
class AffineCoupling(layers.Layer):
    def __init__(self):
        super(AffineCoupling, self).__init__()

    def call(self, x):
        x1, x2 = tf.split(x, num_or_size_splits=2, axis=-1)
        shift = tf.nn.tanh(x1)
        log_scale = tf.nn.tanh(x1)
        y1 = x1
        y2 = x2 * tf.exp(log_scale) + shift
        return tf.concat([y1, y2], axis=-1)

    def inverse(self, y):
        y1, y2 = tf.split(y, num_or_size_splits=2, axis=-1)
        shift = tf.nn.tanh(y1)
        log_scale = tf.nn.tanh(y1)
        x1 = y1
        x2 = (y2 - shift) / tf.exp(log_scale)
        return tf.concat([x1, x2], axis=-1)

# Define the RealNVP model with multiple affine coupling layers
class RealNVPModule(keras.Model):
    def __init__(self):
        super(RealNVPModule, self).__init__()
        self.layers_ = [AffineCoupling() for _ in range(4)]  # Stack 4 layers

    def call(self, x):
        for layer in self.layers_:
            x = layer(x)
        return x

    def inverse(self, y):
        for layer in reversed(self.layers_):
            y = layer.inverse(y)
        return y

# Create the RealNVP model
realnvp_model = RealNVPModule()

# Forward pass (data transformation)
x = tf.random.normal((32, 28, 28, 1))  # Batch of 32 samples
y = realnvp_model(x)

# Inverse pass (sampling)
x_reconstructed = realnvp_model.inverse(y)


# 3. Glow (Invertible 1x1 Convolutions)


In [ ]:
# Define the 1x1 invertible convolution layer for Glow
class Invertible1x1Conv(layers.Layer):
    def __init__(self, num_channels):
        super(Invertible1x1Conv, self).__init__()
        w_init = tf.random.normal([num_channels, num_channels])
        self.w = tf.Variable(w_init, trainable=True)

    def call(self, x):
        shape = tf.shape(x)
        x = tf.reshape(x, [-1, shape[-1]])
        y = tf.matmul(x, self.w)
        return tf.reshape(y, shape)

    def inverse(self, y):
        shape = tf.shape(y)
        y = tf.reshape(y, [-1, shape[-1]])
        w_inv = tf.linalg.inv(self.w)
        x = tf.matmul(y, w_inv)
        return tf.reshape(x, shape)

# Define the Glow model with invertible 1x1 convolutions
class GlowModule(keras.Model):
    def __init__(self, num_channels):
        super(GlowModule, self).__init__()
        self.layers_ = [Invertible1x1Conv(num_channels) for _ in range(4)]  # Stack 4 layers

    def call(self, x):
        for layer in self.layers_:
            x = layer(x)
        return x

    def inverse(self, y):
        for layer in reversed(self.layers_):
            y = layer.inverse(y)
        return y

# Create the Glow model
glow_model = GlowModule(num_channels=28*28)  # Example for image size

# Forward pass (data transformation)
x = tf.random.normal((32, 28, 28, 1))  # Batch of 32 samples
y = glow_model(x)

# Inverse pass (sampling)
x_reconstructed = glow_model.inverse(y)


# 1. RealNVP (Using TensorFlow Probability)


In [13]:
!pip install tensorflow-probability

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

def create_realnvp_model(input_shape):
    # Define base distribution (e.g., standard normal)
    base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros(input_shape))

    # Define bijectors (RealNVP layers)
    bijectors = []
    for i in range(2):  # Example with 2 RealNVP layers
        bijectors.append(tfb.RealNVP(
            num_masked=input_shape // 2,
            shift_and_log_scale_fn=tfb.real_nvp_default_template(
                hidden_layers=[512, 512]
            )
        ))
        bijectors.append(tfb.Permute(permutation=[i for i in range(input_shape)])) # Permutation after each RealNVP

    # Chain bijectors
    bijector = tfb.Chain(bijectors)

    # Create transformed distribution
    return tfd.TransformedDistribution(
        distribution=base_dist,
        bijector=bijector
    )


    return realnvp_model

# Apply the RealNVP model
input_shape = 28 * 28  # Example for MNIST images (flattened)
realnvp_model = create_realnvp_model(input_shape)

# Example usage: Generate data
samples = realnvp_model.sample(10)  # Generate 10 samples
log_prob = realnvp_model.log_prob(samples)  # Calculate log likelihood
print("Generated Samples: ", samples)
print("Log Probability: ", log_prob)

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/bijectors/real_nvp.py:393: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  x = tf_keras.tf1_layers.dense(
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/bijectors/real_nvp.py:399: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  x = tf_keras.tf1_layers.dense(


Generated Samples:  tf.Tensor(
[[-0.82689685 -0.35740632 -0.28841424 ...  0.2582227  -1.4775872
   1.4802727 ]
 [-0.2492197  -0.90530276 -0.47811446 ...  0.97475153 -0.20223337
   1.3833971 ]
 [-0.23200765  0.33753997 -0.6723876  ...  0.6379681   0.24341452
   2.7222257 ]
 ...
 [ 1.3190647  -0.12942575 -1.1176198  ...  0.73666245  0.1583738
   0.9330384 ]
 [ 0.5885623  -0.60022354 -0.44780692 ... -0.22298717 -0.20464176
  -0.18658447]
 [-0.31655037  0.99625     1.8196329  ... -0.06047153 -0.31841826
   2.2305033 ]], shape=(10, 784), dtype=float32)
Log Probability:  tf.Tensor(
[-1115.6232 -1115.0029 -1123.6189 -1115.2931 -1154.9408 -1109.9989
 -1106.2806 -1146.9216 -1112.8838 -1104.9581], shape=(10,), dtype=float32)


#  2. Glow (Using TensorFlow Probability)


In [14]:
class InvertibleConv(tf.keras.layers.Layer):
    def __init__(self, num_channels):
        super(InvertibleConv, self).__init__()
        w_init = tf.random.normal([num_channels, num_channels])
        self.w = tf.Variable(w_init, trainable=True)

    def call(self, x):
        shape = tf.shape(x)
        x = tf.reshape(x, [-1, shape[-1]])
        y = tf.matmul(x, self.w)
        return tf.reshape(y, shape)

    def inverse(self, y):
        shape = tf.shape(y)
        y = tf.reshape(y, [-1, shape[-1]])
        w_inv = tf.linalg.inv(self.w)
        x = tf.matmul(y, w_inv)
        return tf.reshape(x, shape)

# Glow Model using Invertible Conv
class Glow(tf.keras.Model):
    def __init__(self, num_channels):
        super(Glow, self).__init__()
        self.invertible_conv = InvertibleConv(num_channels)

    def call(self, x):
        return self.invertible_conv(x)

    def inverse(self, y):
        return self.invertible_conv.inverse(y)

# Use Glow model
glow_model = Glow(num_channels=28*28)  # Example for MNIST
x = tf.random.normal((32, 28*28))  # Batch of 32 samples
y = glow_model(x)
x_reconstructed = glow_model.inverse(y)


# Steps to Apply on Real Problems:


# 1. Image Generation (MNIST Example):


In [15]:
# Use real-world dataset (MNIST in this case)
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(-1, 28*28).astype('float32') / 255.0

# Define RealNVP model
realnvp_model = create_realnvp_model(input_shape=28*28)

# **Access trainable variables directly, not through 'layers'**
for variable in realnvp_model.trainable_variables:
    variable.trainable = True  # This line might be redundant, but ensures trainability

# Train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Training loop (simple example)
for epoch in range(10):
    with tf.GradientTape() as tape:
        # Random batch from dataset
        batch = tf.random.shuffle(train_images)[:32]
        neg_log_likelihood = -realnvp_model.log_prob(batch)  # Compute negative log-likelihood
        loss = tf.reduce_mean(neg_log_likelihood)

    gradients = tape.gradient(loss, realnvp_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, realnvp_model.trainable_variables))
    print(f'Epoch {epoch+1}, Loss: {loss.numpy()}')

# Generate new samples
generated_samples = realnvp_model.sample(10)

ValueError: not enough values to unpack (expected 2, got 0)

# 2. Anomaly Detection:


In [16]:
# Assume `test_data` is new data where we want to detect anomalies
test_data = tf.random.normal((32, 28*28))

# Calculate log-probability for each point
log_probs = realnvp_model.log_prob(test_data)

# Anomalies: Points with log-prob below a threshold
threshold = -10.0  # Define a threshold for low probability
anomalies = log_probs < threshold
print("Anomalies Detected: ", anomalies)


Anomalies Detected:  tf.Tensor(
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True], shape=(32,), dtype=bool)
